## Load Data

In [2]:
import os, time, json
import kagglehub
import optuna
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBClassifier



path = kagglehub.dataset_download("naserabdullahalam/phishing-email-dataset")
data = []

datasets = [
    "Nigerian_Fraud.csv",
    "Ling.csv",
    "Nazario.csv",
    "SpamAssasin.csv",
    "CEAS_08.csv",
    # "Enron.csv"
 ]
for file in datasets:
    print(file)
    csv_path = os.path.join(path, file)
    subset_data = pd.read_csv(csv_path)
    data.append(subset_data)

all_data = pd.concat(data)


Nigerian_Fraud.csv
Ling.csv
Nazario.csv
SpamAssasin.csv
CEAS_08.csv


## Clean & Split Data

In [3]:
all_data['receiver'] = all_data['receiver'].str.replace('undisclosed-recipients:;', 'Unknown')
# -- Timestamp features -- 
all_data['date_parsed'] = pd.to_datetime(all_data['date'], errors='coerce', utc=True)

# Week-of-year
iso_week = all_data['date_parsed'].dt.isocalendar().week
iso_week = iso_week.astype(float)
week0 = ((iso_week - 1) % 52)
theta_week = 2.0 * np.pi * week0 / 52
all_data['sin_week'] = np.where(week0.notna(), np.sin(theta_week), 0.0)
all_data['cos_week'] = np.where(week0.notna(), np.cos(theta_week), 0.0)

# Hour-of-day 
hour = all_data['date_parsed'].dt.hour.astype(float)  # NaN for missing
theta_hour = 2.0 * np.pi * hour / 24
all_data['sin_hour'] = np.where(hour.notna(), np.sin(theta_hour), 0.0)
all_data['cos_hour'] = np.where(hour.notna(), np.cos(theta_hour), 0.0)

# Weekend binary (0/1)
weekday = all_data['date_parsed'].dt.weekday
all_data['is_weekend'] = np.where(weekday.isna(), 0, ((weekday >= 5).astype(int)))

# Timestamp feature list
timestamp_features = [
    "sin_week",
    "cos_week",
    "sin_hour",
    "cos_hour",
    "is_weekend"
]

# -- Sender/reciever feature engineering -- 
with open('domains.json', 'r') as file:
    public_email_domains = json.load(file)
    
email_regex = r'([a-zA-Z0-9._%+\-|{}^&"\'=]+@(?:[a-zA-Z0-9.-]+|\[[0-9.]+\]))'    
for column_name in ('sender', 'receiver'):
    all_data[f'{column_name}_email'] = all_data[column_name].str.extract(email_regex, expand=False)
    all_data[f'{column_name}_domain'] = all_data[f'{column_name}_email'].str.split('@', n=1).str[1]
    all_data[f'{column_name}_domain_len'] = all_data[f'{column_name}_domain'].str.len()
    all_data[f'{column_name}_domain_public'] = all_data[f'{column_name}_domain'].str.lower().isin(public_email_domains).astype(int)
    all_data[f'{column_name}_n_subdomains'] = all_data[f'{column_name}_domain'].str.lower().str.count(r'\.')
    all_data[f'{column_name}_email_n_digits'] = all_data[f'{column_name}_domain'].str.lower().str.count(r'\d')
    
    all_data[f'{column_name}_name'] = all_data[column_name].str.replace(email_regex, '', regex=True)
    all_data[f'{column_name}_name'] = all_data[f'{column_name}_name'].str.replace(r'[<>"\'\(\)]', '', regex=True).str.strip()
    
all_data['is_internal_email'] = (
    (all_data['sender_domain'] == all_data['receiver_domain']) & 
    (all_data['sender_domain'].notna())
).astype(int)

all_data['sender_name_contains_email'] = all_data['sender_name'].str.contains('@', na=False).astype(int)

# Sender/reciever feature list
email_features = [
    "sender_domain_public",
    "sender_domain_len",
    "sender_n_subdomains",
    "sender_email_n_digits",
    "sender_name_contains_email",
    # "is_internal_email"
]

# -- Fill in url count for missing entries -- 
url_regex = r'((?:https?|ftp)://\S+|www\.\S+)'
text_column = 'body' 
missing_count_mask = all_data['urls'].isna()
all_data.loc[missing_count_mask, 'urls'] = (
    all_data.loc[missing_count_mask, text_column]
    .astype(str)
    .str.count(url_regex)
)

all_data[['body', 'subject']] = all_data[['body', 'subject']].fillna('Unknown')

feature_set= [
    'subject',
    'body',
    *email_features,
    *timestamp_features
]

X = all_data[feature_set]
y = all_data['label'] 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [4]:
# -- Initialize Model --
vectorizer = ColumnTransformer([
    ('subject_word', TfidfVectorizer(lowercase=False, analyzer='word'), 'subject'),
    ('subject_charwb', TfidfVectorizer(lowercase=False, analyzer='char_wb'), 'subject'),
    ('body', TfidfVectorizer(lowercase=True, analyzer='word'), 'body' )
    ],
    remainder="passthrough"
)

model_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('xgboost', XGBClassifier())
        
])

In [5]:
# -- Hyperparameter tuning with Optuna -- 
GPU = True
def objective(trial):
    start_time = time.time()
    
    # Define Search Space
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
    max_depth = trial.suggest_int('max_depth', 5, 12)
    subsample = trial.suggest_float('subsample', 0.15, .9)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    
    ngram_choice = trial.suggest_categorical('ngram_range', ['unigram', 'bigram'])
    ngram_range = (1, 1) if ngram_choice == 'unigram' else (1, 2)
    stop_words_choice = trial.suggest_categorical('stop_words_setting', ['english', 'none'])
    stop_words = 'english' if stop_words_choice == 'english' else None
    df_min_type = trial.suggest_categorical('df_min_type', ['int', 'float'])


    max_features = None
    if df_min_type == 'int':
        min_df = trial.suggest_int('df_min', 2, 20)
    elif df_min_type == 'float':
        min_df = trial.suggest_float('min_df', 0.0001, 0.3, log=True)

    
    max_features = trial.suggest_int('max_features', 10000, 200000) if min_df < 0.01 or min_df == 2 else None

    
    # Build Vectorizer
    vectorizer = ColumnTransformer([
        ('subject_word_vectorizer', TfidfVectorizer(lowercase=False, analyzer='word', ngram_range=ngram_range), 'subject'),
        ('subject_charwb_vectorizer', TfidfVectorizer(lowercase=False, analyzer='char_wb'), 'subject'),
        ('body_vectorizer', TfidfVectorizer(
            lowercase=True, 
            analyzer='word', 
            min_df=min_df, 
            ngram_range=ngram_range,
            stop_words=stop_words,
            max_features=max_features
        ), 'body')
        ],
        remainder="passthrough"
    )
    
    # Train/Validation Split
    X_sub_train, X_sub_valid, y_sub_train, y_sub_valid = train_test_split(
        X_train,
        y_train,
        test_size=0.2,
        stratify=y_train
    )
    vectorizer.fit(X_sub_train)
    X_sub_train_vec = vectorizer.transform(X_sub_train)
    X_sub_valid_vec = vectorizer.transform(X_sub_valid)
    
    # Model with pruning + early stopping
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-auc")
    
    model = XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        n_jobs=-1,
        tree_method='hist',
        device='cuda' if GPU else 'cpu',
        objective='binary:logistic',
        eval_metric='auc',
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )
    
    model.fit(
        X_sub_train_vec,
        y_sub_train,
        eval_set=[(X_sub_valid_vec, y_sub_valid)],
        verbose=False
    )
    
    preds = model.predict(X_sub_valid_vec)
    f1 = f1_score(y_sub_valid, preds)
    elapsed = time.time() - start_time
    print(f"Trial {trial.number} done in {elapsed:.2f}s | F1: {f1:.4f}")
    return f1
    
start = time.perf_counter()
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_startup_trials=20, n_warmup_steps=20, interval_steps=100))

study.optimize(objective, n_trials=100, n_jobs=1)

end = time.perf_counter()
duration = end - start
print(f"Study took: {duration//(60**2)}m {duration//60}m {round(duration%60)}s")
print(f"Best value: {study.best_value}")
print(f"Best params: {study.best_params}")

[I 2025-12-10 17:05:17,710] A new study created in memory with name: no-name-ed3e75b2-4897-4610-b35f-acabeebda44b
/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [17:05:27] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 17:07:01,450] Trial 0 finished with value: 0.9812831331818673 and parameters: {'n_estimators': 116, 'learning_rate': 0.0598121908100976, 'max_depth': 12, 'subsample': 0.6042214363162028, 'colsample_bytree': 0.7348192358510411, 'ngram_range': 'bigram', 'stop_words_setting': 'english', 'df_min_type': 'float', 'min_df': 0.09676214928169895}. Best is trial 0 with value: 0.9812831331818673.


Trial 0 done in 103.73s | F1: 0.9813


/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [17:07:08] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 17:10:09,528] Trial 1 finished with value: 0.9933903998266335 and parameters: {'n_estimators': 703, 'learning_rate': 0.15965232560070328, 'max_depth': 6, 'subsample': 0.8363550644662354, 'colsample_bytree': 0.5919439430073383, 'ngram_range': 'unigram', 'stop_words_setting': 'none', 'df_min_type': 'float', 'min_df': 0.008151902229152496, 'max_features': 112776}. Best is trial 1 with value: 0.9933903998266335.


Trial 1 done in 188.08s | F1: 0.9934


/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [17:10:19] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 17:16:30,655] Trial 2 finished with value: 0.9794594594594594 and parameters: {'n_estimators': 715, 'learning_rate': 0.2592395681103579, 'max_depth': 6, 'subsample': 0.24775210554362517, 'colsample_bytree': 0.6841908805653953, 'ngram_range': 'bigram', 'stop_words_setting': 'english', 'df_min_type': 'float', 'min_df': 0.10492651066689693}. Best is trial 1 with value: 0.9933903998266335.


...

Trial 94 done in 124.02s | F1: 0.9932


/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [23:17:38] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 23:19:17,358] Trial 95 finished with value: 0.9934980494148244 and parameters: {'n_estimators': 744, 'learning_rate': 0.26027587113911094, 'max_depth': 10, 'subsample': 0.8107454691899771, 'colsample_bytree': 0.7648901584591274, 'ngram_range': 'unigram', 'stop_words_setting': 'none', 'df_min_type': 'int', 'df_min': 17}. Best is trial 83 with value: 0.9952308692824626.


Trial 95 done in 104.55s | F1: 0.9935


/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [23:19:23] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 23:19:51,369] Trial 96 pruned. Trial was pruned at iteration 20.
/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [23:19:58] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
[I 2025-12-10 23:20:16,385] Trial 97 pruned. Trial was pruned at iteration 20.
/Users/james/Documents/phishguard/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [23:20:23] WARNING: /Users/runner/work/xgboost/xgboost/src/context.cc:207: Device is changed fr

Trial 99 done in 98.72s | F1: 0.9926
Study took: 6.0m 362.0m 17s
Best value: 0.9952308692824626
Best params: {'n_estimators': 804, 'learning_rate': 0.17389030487747995, 'max_depth': 9, 'subsample': 0.8405243248517169, 'colsample_bytree': 0.6457296831288679, 'ngram_range': 'unigram', 'stop_words_setting': 'none', 'df_min_type': 'int', 'df_min': 19}


In [8]:
importances = optuna.importance.get_param_importances(study)

# Print out the dictionary nicely
for param, score in importances.items():
    print(f"{param}: {score:.4f} ({score*100:.2f}%)")


# --- PART 2: Top 10 Models (Trials) ---
print("\n--- Top 10 Models ---")
# Convert the study to a Pandas DataFrame for easy sorting/viewing
df = study.trials_dataframe()

# Sort by objective value (ascending for minimization) and take top 10
# We filter columns to only show params and the resulting value
cols_to_keep = ['value'] + [c for c in df.columns if c.startswith('params_')]
top_10 = df.sort_values('value', ascending=False).head(10)[cols_to_keep]
print(top_10)

subsample: 0.4941 (49.41%)
n_estimators: 0.2525 (25.25%)
stop_words_setting: 0.0808 (8.08%)
learning_rate: 0.0705 (7.05%)
max_depth: 0.0509 (5.09%)
df_min_type: 0.0290 (2.90%)
colsample_bytree: 0.0117 (1.17%)
ngram_range: 0.0104 (1.04%)

--- Top 10 Models ---
       value  params_colsample_bytree  params_df_min params_df_min_type  \
93  0.999502                 0.659702           16.0                int   
84  0.999486                 0.664306           19.0                int   
26  0.999451                 0.634545            9.0                int   
72  0.999389                 0.565086            NaN              float   
87  0.999379                 0.715503           20.0                int   
42  0.999221                 0.632838           17.0                int   
89  0.998377                 0.702237           20.0                int   
98  0.998272                 0.634886           16.0                int   
96  0.998266                 0.576372           20.0             